In [30]:
# FM
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# GPU 확인
tf.config.list_physical_devices('GPU')

# 자료형 선언
tf.keras.backend.set_floatx('float32')


In [10]:

#### 마스터 데이터(상호 작용)
#### 상호 작용(transaction) 마스터 데이터를 불러옴
masterdf = pd.read_csv('./data/Transactions.csv')
masterdf.head()

### 데이터 정리 및 표준화를 위해 데이터 열 명칭 변경
### 표준화는 병합의 용의성을 위해 열 명칭을 정렬
masterdf.columns = ['Transaction ID', 'Customer ID', 'Transaction Date', 'Prod Subcat Code',
       'Prod Cat Code', 'Qty', 'Rate', 'Tax', 'Total Amt', 'Store Type']

###  상점 코드 타입을 숫자형으로 변경하여 새 열에 저장
masterdf['Store Type Code'] = pd.factorize(masterdf['Store Type'])[0]
masterdf.head(5)

### quantity와 based price에서 총 순 매출액(Net sales) 계산 (도시마다의 세금이 다를 수 있어 세금 제외)
masterdf['Net Sales'] = masterdf['Qty'] * masterdf['Rate']

### category, subcategory, store type을 이용하여 고유한 material 표시기를 생성
### 다른 sku는 다른 상점 유형에 판매된다고 가정
masterdf['Material'] = masterdf['Prod Cat Code'].astype(str) + '-' + masterdf['Prod Subcat Code'].astype(str) + '-' + masterdf['Store Type'].astype(str)

masterdf[['Customer ID','Material','Net Sales']]

scaler = MinMaxScaler()
grade = scaler.fit_transform(masterdf[['Net Sales']])

grade = pd.DataFrame(grade,columns=['grade'])
cust = pd.get_dummies(masterdf['Customer ID'],prefix='cust')
item = pd.get_dummies(masterdf['Material'],prefix='item')

df = pd.concat([cust,item,grade], axis=1)

In [11]:
# 데이터 로드
X, Y = df[[i for i in df.columns if i!='grade']].to_numpy(), df[['grade']].to_numpy()
X = X
Y = Y

n = X.shape[0]
p = X.shape[1]

k = 10
batch_size = 32
epochs = 100

In [12]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [13]:
Y

array([[0.51009421],
       [0.99798116],
       [0.20349933],
       ...,
       [0.55693136],
       [0.45168237],
       [0.05074024]])

# batch train style

In [14]:
class FM(tf.keras.Model):
    def __init__(self):
        super(FM, self).__init__()

        # 모델의 파라미터 정의
        self.w_0 = tf.Variable([0.0])
        self.w = tf.Variable(tf.zeros([p]))
        self.V = tf.Variable(tf.random.normal(shape=(p, k)))

    def call(self, inputs):
        linear_terms = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1)

        interactions = 0.5 * tf.reduce_sum(
            tf.math.pow(tf.matmul(inputs, self.V), 2)
            - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.V, 2)),
            1,
            keepdims=False
        )

        y_hat = tf.math.sigmoid(self.w_0 + linear_terms + interactions)

        return y_hat

In [15]:
# Forward
def train_on_batch(model, optimizer, accuracy, inputs, targets):
    with tf.GradientTape() as tape:
        y_pred = model(inputs)
        loss = tf.keras.losses.mean_squared_error(y_true=targets, y_pred=y_pred)
    
    # loss를 모델의 파라미터로 편미분하여 gradients를 구한다.
    grads = tape.gradient(target=loss, sources=model.trainable_variables)

    # apply_gradients()를 통해 processed gradients를 적용한다.
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # accuracy: update할 때마다 정확도는 누적되어 계산된다.
    accuracy.update_state(targets, y_pred)

    return loss


# 반복 학습 함수
def train(epochs):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

    train_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_train, tf.float32), tf.cast(Y_train, tf.float32))).shuffle(500,seed = 42).batch(batch_size)

    test_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_test, tf.float32), tf.cast(Y_test, tf.float32))).shuffle(200,seed = 42).batch(batch_size)

    model = FM()
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    accuracy = MeanAbsoluteError()
    loss_history = []

    for i in range(epochs):
        for x, y in train_ds:
            loss = train_on_batch(model, optimizer, accuracy, x, y)
            if len(loss) == batch_size:
                loss_history.append(loss)

        if i % 2== 0:
            print("스텝 {:03d}에서 누적 평균 손실: {:.4f}".format(i, np.mean(loss_history)))
            print("스텝 {:03d}에서 누적 정확도: {:.4f}".format(i, accuracy.result().numpy()))

    test_accuracy = MeanAbsoluteError()
    for x, y in test_ds:
        y_pred = model(x)
        test_accuracy.update_state(y, y_pred)

    print("테스트 정확도: {:.4f}".format(test_accuracy.result().numpy()))

In [16]:
train(epochs)

스텝 000에서 누적 평균 손실: 0.1621
스텝 000에서 누적 정확도: 0.3154
스텝 002에서 누적 평균 손실: 0.1483
스텝 002에서 누적 정확도: 0.2989
스텝 004에서 누적 평균 손실: 0.1446
스텝 004에서 누적 정확도: 0.2945
스텝 006에서 누적 평균 손실: 0.1425
스텝 006에서 누적 정확도: 0.2919
스텝 008에서 누적 평균 손실: 0.1408
스텝 008에서 누적 정확도: 0.2900
스텝 010에서 누적 평균 손실: 0.1393
스텝 010에서 누적 정확도: 0.2883
스텝 012에서 누적 평균 손실: 0.1380
스텝 012에서 누적 정확도: 0.2867
스텝 014에서 누적 평균 손실: 0.1368
스텝 014에서 누적 정확도: 0.2852
스텝 016에서 누적 평균 손실: 0.1356
스텝 016에서 누적 정확도: 0.2838
스텝 018에서 누적 평균 손실: 0.1344
스텝 018에서 누적 정확도: 0.2824
스텝 020에서 누적 평균 손실: 0.1332
스텝 020에서 누적 정확도: 0.2809
스텝 022에서 누적 평균 손실: 0.1320
스텝 022에서 누적 정확도: 0.2795
스텝 024에서 누적 평균 손실: 0.1308
스텝 024에서 누적 정확도: 0.2780
스텝 026에서 누적 평균 손실: 0.1295
스텝 026에서 누적 정확도: 0.2765
스텝 028에서 누적 평균 손실: 0.1281
스텝 028에서 누적 정확도: 0.2749
스텝 030에서 누적 평균 손실: 0.1267
스텝 030에서 누적 정확도: 0.2733
스텝 032에서 누적 평균 손실: 0.1253
스텝 032에서 누적 정확도: 0.2716
스텝 034에서 누적 평균 손실: 0.1238
스텝 034에서 누적 정확도: 0.2699
스텝 036에서 누적 평균 손실: 0.1223
스텝 036에서 누적 정확도: 0.2681
스텝 038에서 누적 평균 손실: 0.1207
스텝 038에서 누적 정확도: 0.2663


# layer train style

## train

In [64]:
model_path = './model/FM.h5'
batch_size = 256
epochs = 700

In [65]:
inputs = keras.Input(shape=(X.shape[1],))
outputs = FM()(inputs)
model = keras.Model(inputs, outputs)

# If there is a loss passed in `compile`, thee regularization
# losses get added to it
model.compile(optimizer="adam", loss="mae")

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(model_path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

model.fit(X_train, Y_train,
        batch_size=batch_size, 
        epochs=epochs, 
        shuffle=True, 
        callbacks=[es, mc], 
        validation_split=0.1)

Epoch 1/700
65/65 [==============================] - 1s 9ms/step - loss: 0.3932 - val_loss: 0.3878

Epoch 00001: val_loss improved from inf to 0.38778, saving model to ./model\FM.h5
Epoch 2/700
65/65 [==============================] - 0s 5ms/step - loss: 0.3852 - val_loss: 0.3813

Epoch 00002: val_loss improved from 0.38778 to 0.38132, saving model to ./model\FM.h5
Epoch 3/700
65/65 [==============================] - 0s 4ms/step - loss: 0.3720 - val_loss: 0.3749

Epoch 00003: val_loss improved from 0.38132 to 0.37486, saving model to ./model\FM.h5
Epoch 4/700
65/65 [==============================] - 0s 4ms/step - loss: 0.3617 - val_loss: 0.3685

Epoch 00004: val_loss improved from 0.37486 to 0.36849, saving model to ./model\FM.h5
Epoch 5/700
65/65 [==============================] - 0s 4ms/step - loss: 0.3462 - val_loss: 0.3625

Epoch 00005: val_loss improved from 0.36849 to 0.36252, saving model to ./model\FM.h5
Epoch 6/700
65/65 [==============================] - 0s 4ms/step - loss: 0

In [66]:
## model load
inputs = keras.Input(shape=(X.shape[1],))
outputs = FM()(inputs)
model = keras.Model(inputs, outputs)
model.load_weights(model_path)
## Validation set Prediction
y_pred=model.predict(X_test)

tmp=[]
for i in range(len(y_pred)):
    tmp.append(abs(Y_test[i]-y_pred[i]))
ANOMALY_SCORE=np.mean(tmp,axis=1)
print(f'{ANOMALY_SCORE.mean()} test MAE')

0.2651144966551641 test MAE


## hyper paraneter tune

In [ ]:
for batch_size in [512,256,128,64,32,16,8]:
    model_path = f'./model/FM_{batch_size}.h5'
    epochs = 700

    inputs = keras.Input(shape=(X.shape[1],))
    outputs = FM()(inputs)
    model = keras.Model(inputs, outputs)

    # If there is a loss passed in `compile`, thee regularization
    # losses get added to it
    model.compile(optimizer="adam", loss="mae")

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    mc = ModelCheckpoint(model_path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

    model.fit(X_train, Y_train,
            batch_size=batch_size, 
            epochs=epochs, 
            shuffle=True, 
            callbacks=[es, mc], 
            validation_split=0.1)

    ## model load
    inputs = keras.Input(shape=(X.shape[1],))
    outputs = FM()(inputs)
    model = keras.Model(inputs, outputs)
    model.load_weights(model_path)
    ## Validation set Prediction
    y_pred=model.predict(X_test)

    tmp=[]
    for i in range(len(y_pred)):
        tmp.append(abs(Y_test[i]-y_pred[i]))
    ANOMALY_SCORE=np.mean(tmp,axis=1)
    print(f'{ANOMALY_SCORE.mean()} test MAE')
    input()